Merge .root files which was created for each sample to a final histograms.root

In [ ]:
import uproot
import json
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
def extract_samples_from_json(json_file):
    output_files = []
    with open(json_file, 'r') as fd:
        data = json.load(fd)
        for sample, conditions in data.items():
            for condition in conditions:
                sample_name = f"everything_merged_{sample}__{condition}.root"
                output_files.append(sample_name)
    return output_files
json_file = "nanoaod_inputs.json"
# LIST_OF_FILES_PER_SAMPLE is a list of 9 files containing histograms per sample
LIST_OF_FILES_PER_SAMPLE = extract_samples_from_json(json_file)
print(LIST_OF_FILES_PER_SAMPLE)
with uproot.recreate("histograms.root") as f_out:
    for h_file in LIST_OF_FILES_PER_SAMPLE:
        with uproot.open(h_file) as f_per_sample:
            for key in f_per_sample.keys(cycle=False):
                value = f_per_sample[key]
                f_out[key] = value

Visualise the created histogram

In [ ]:
def plot_stacked_histograms(file_path, tag):
    file = uproot.open(file_path)
    processes = ['ttbar', 'wjets', 'single_top_tW', 'single_top_t_chan', 'single_top_s_chan']
    hist_dict = {}
    for process in processes:
        key = f"{tag}_{process};1" 
        hist = file[key]
        # Storing values and bin edges
        hist_dict[process] = hist.values(), hist.axes[0].edges()
    # Initialized an array of zeros that has the same shape as the first histogram's values.
    stacked_values = np.zeros_like(hist_dict[processes[0]][0])
    plt.figure()
    # Plot the stacked histograms in reverse order to stack them correctly
    for process in processes[::-1]: 
        values, bin_edges = hist_dict[process]
        plt.hist(bin_edges[:-1], bins=bin_edges, weights=values + stacked_values, 
                 histtype='stepfilled', alpha=0.6, label=process)
        stacked_values += values 
    plt.legend(frameon=False)
    if tag == "4j2b":
        plt.title("$\geq$ 4 jets, $\geq$ 2 b-tags")
    elif tag == "4j1b":
        plt.title("$\geq$ 4 jets, $\geq$ 1 b-tags")
    plt.xlabel("$m_{bjj}$ [GeV]")
    plt.ylabel("Events")
    plt.show()
file_path = "histograms.root"
plot_stacked_histograms(file_path, "4j2b")
plot_stacked_histograms(file_path, "4j1b")